In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os 
import sys 
import torch 
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import lightning.pytorch as pl
import wandb
import numpy as np 
import matplotlib.pyplot as plt
from lightning.pytorch.loggers import WandbLogger


sys.path.append('/Users/ivan_zorin/Documents/DEV/code/ntl/')

from models import *
from data.data import SGCCDataset, sgcc_train_test_split
from utils.utils import *

from tqdm.auto import tqdm 
import models

In [20]:
# config and logger
pathes_file = '../configs/local_pathes.yaml'
config_path = '../configs/config.yaml'

config = load_config(config_path, pathes_file)
wandb_logger = WandbLogger(**config.logger) # TODO move this inside LightningModule
# wandb.save(config_path) # save config file

# random seed
if config.seed is not None:
    pl.seed_everything(config.seed, workers=True)

/Users/ivan_zorin/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/lightning/pytorch/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Global seed set to 42


In [35]:
wandb.run.dir # TODO use this to init tensorboard experiment

'/Users/ivan_zorin/Documents/DEV/runs/wandb/offline-run-20230602_114942-qrg7d6kc/files'

In [ ]:
# data
train_data, val_data, test_data = sgcc_train_test_split(config)
num_workers = config.num_workers
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, num_workers=num_workers)

if config.supervised_validation:
    val_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False,  num_workers=num_workers)
    
else:
    val_loader = DataLoader(val_data, batch_size=config.batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_data, batch_size=config.batch_size, shuffle=False,  num_workers=num_workers)



In [ ]:
# base model
nn_model = getattr(models, config.model)
nn_model = nn_model(**config.model_kwargs)
# Lightning model
model = LitLSTMAE(
    model=nn_model,
    loss_fn=getattr(torch.nn.functional, config.loss),
    optimizer=getattr(torch.optim, config.optimizer),
    scheduler=getattr(torch.optim.lr_scheduler, config.scheduler),
    config=config
)


In [ ]:
# Trainer
trainer = pl.Trainer(accelerator=config.device,
                        logger=wandb_logger,
                        
                        **config.trainer_kwargs

                        
)


In [ ]:
# fit loop
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

In [ ]:
# test() if validation was unsupervised
if not config.supervised_validation:
    trainer.test(model, test_loader)



In [ ]:
# upload logs
model.clear_mem()
wandb.finish()

In [3]:
N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N) * 30

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    marker=go.scatter.Marker(
        size=sz,
        color=colors,
        opacity=0.6,
        colorscale="Viridis"
    )
))

In [10]:
temp = fig.to_image(format='svg', )

In [11]:
import PIL as pil

In [32]:
im = pil.Image.frombytes('RGB', size=(200, 200), data=temp)

ValueError: not enough image data

In [29]:
im.show()